In [1]:
%pip install -U openmim
!mim install mmcv-full

# # install mmpose dependencies
# %pip install -v -r requirements.txt
# %pip install mmdet
# # install mmpose in develop mode
%pip install -v -e .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 13.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
/usr/local/lib/python3.9/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.12.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 7.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 kB 24.4 MB/s eta 0:00:00
Using pip 22.2.2 from /usr/local/lib/python3.9/dist-packages/pip (python 3.9)
Obtaining file:///notebooks
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-qu4bej3k/mmdet.egg-info
  writing /tmp/pip-pip-egg-info-qu4

In [2]:
#Add custom dataset
%run mmdet/datasets/acino_det.py

In [3]:
# Copyright (c) OpenMMLab. All rights reserved.
import argparse
import copy
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
import torch.distributed as dist
from mmcv import Config, DictAction
from mmcv.runner import get_dist_info, init_dist
from mmcv.utils import get_git_hash

from mmdet import __version__
from mmdet.apis import init_random_seed, set_random_seed, train_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.utils import (collect_env, get_device, get_root_logger,
                         replace_cfg_vals, setup_multi_processes,
                         update_data_root)

def train_from_config(config,eps=5):
    cfg = Config.fromfile(config) 
    # replace the ${key} with the value of cfg.key
    cfg = replace_cfg_vals(cfg)
    
    # update data root according to MMDET_DATASETS
    update_data_root(cfg)
    
    cfg.lr_config.policy='exp'
    cfg.lr_config.gamma=0.8
    cfg.optimizer.lr=1e-3
    # print(cfg.lr_config)
    # set multi-process settings
    setup_multi_processes(cfg)
    cfg.work_dir = osp.join('./work_dirs',
                                osp.splitext(osp.basename(config))[0])
    
    cfg.gpu_ids = range(1)
    cfg.checkpoint_config.max_keep_ckpts=1
    distributed = False
    cfg.total_epochs=eps
    cfg.runner.max_epochs=cfg.total_epochs
    # create work_dir
    mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
    # dump config
    cfg.dump(osp.join(cfg.work_dir, osp.basename(config)))
    # init the logger before other steps
    timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
    logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)
    
    # init the meta dict to record some important information such as
    # environment info and seed, which will be logged
    meta = dict()
    # log env info
    env_info_dict = collect_env()
    env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])
    dash_line = '-' * 60 + '\n'
    logger.info('Environment info:\n' + dash_line + env_info + '\n' +
                dash_line)
    meta['env_info'] = env_info
    # log some basic info
    logger.info(f'Distributed training: {distributed}')

    cfg.device = get_device()
    # set random seeds
    seed = init_random_seed(0, device=cfg.device)
    # seed = seed + dist.get_rank()
    logger.info(f'Set random seed to {seed}, '
                f'deterministic: {True}')
    set_random_seed(seed, deterministic=True)
    cfg.seed = seed
    meta['seed'] = seed
    meta['exp_name'] = osp.basename(config)

    model = build_detector(
        cfg.model,
        train_cfg=cfg.get('train_cfg'),
        test_cfg=cfg.get('test_cfg'))
    model.init_weights()

    datasets = [build_dataset(cfg.data.train)]
    model.CLASSES = datasets[0].CLASSES
    train_detector(
        model,
        datasets,
        cfg,
        distributed=distributed,
        validate=True,
        timestamp=timestamp,
        meta=meta)

In [39]:
train_from_config('configs/retinanet/retinanet_r50_fpn_2x_acino.py',eps=10)

2022-10-16 15:07:46,363 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.9.13 (main, May 23 2022, 22:01:06) [GCC 9.4.0]
CUDA available: True
GPU 0: NVIDIA RTX A4000
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.2, V11.2.152
GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
PyTorch: 1.12.0+cu116
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=c

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2022-10-16 15:08:08,373 - mmdet - INFO - Epoch [1][50/1337]	lr: 9.890e-05, eta: 1:34:20, time: 0.425, data_time: 0.057, memory: 14415, loss_cls: 1.1326, loss_bbox: 0.6740, loss: 1.8065
2022-10-16 15:08:27,524 - mmdet - INFO - Epoch [1][100/1337]	lr: 1.988e-04, eta: 1:29:20, time: 0.383, data_time: 0.010, memory: 14415, loss_cls: 1.1306, loss_bbox: 0.6517, loss: 1.7823
2022-10-16 15:08:47,310 - mmdet - INFO - Epoch [1][150/1337]	lr: 2.987e-04, eta: 1:28:24, time: 0.396, data_time: 0.019, memory: 14415, loss_cls: 1.1237, loss_bbox: 0.6255, loss: 1.7492
2022-10-16 15:09:06,748 - mmdet - INFO - Epoch [1][200/1337]	lr: 3.986e-04, eta: 1:27:22, time: 0.389, data_time: 0.011, memory: 14415, loss_cls: 1.0629, loss_bbox: 0.6092, loss: 1.6721
2022-10-16 15:09:26,230 - mmdet - INFO - Epoch [1][250/1337]	lr: 4.985e-04, eta: 1:26:40, time: 0.390, data_time: 0.011, memory: 14415, loss_cls: 0.5435, loss_bbox: 0.5456, loss: 1.0890
2022-10-16 15:09:45,751 - mmdet - INFO - Epoch [1][300/1337]	lr: 5.984e

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 764/764, 19.9 task/s, elapsed: 38s, ETA:     0s

2022-10-16 15:17:13,823 - mmdet - INFO - Evaluating bbox...


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 764/764, 19.8 task/s, elapsed: 39s, ETA:     0s

2022-10-16 16:42:33,520 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


2022-10-16 16:42:33,897 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.736
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.978
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.894
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.520
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.748
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.790
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.790
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.790
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.589
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.26s).
Accumulating evaluation results...
DONE (t=0.09s).


In [41]:
train_from_config('configs/faster_rcnn/faster_rcnn_r50_fpn_2x_acino.py',eps=10)

2022-10-16 16:42:36,625 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.9.13 (main, May 23 2022, 22:01:06) [GCC 9.4.0]
CUDA available: True
GPU 0: NVIDIA RTX A4000
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.2, V11.2.152
GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
PyTorch: 1.12.0+cu116
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=c

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2022-10-16 16:43:02,966 - mmdet - INFO - Epoch [1][50/1337]	lr: 9.890e-05, eta: 1:52:38, time: 0.507, data_time: 0.058, memory: 14415, loss_rpn_cls: 0.6780, loss_rpn_bbox: 0.0088, loss_cls: 0.2721, acc: 89.8486, loss_bbox: 0.0151, loss: 0.9740
2022-10-16 16:43:26,335 - mmdet - INFO - Epoch [1][100/1337]	lr: 1.988e-04, eta: 1:47:47, time: 0.467, data_time: 0.011, memory: 14415, loss_rpn_cls: 0.4202, loss_rpn_bbox: 0.0106, loss_cls: 0.0444, acc: 99.6309, loss_bbox: 0.0086, loss: 0.4838
2022-10-16 16:43:50,477 - mmdet - INFO - Epoch [1][150/1337]	lr: 2.987e-04, eta: 1:47:03, time: 0.483, data_time: 0.018, memory: 14415, loss_rpn_cls: 0.0430, loss_rpn_bbox: 0.0098, loss_cls: 0.0682, acc: 98.9463, loss_bbox: 0.0367, loss: 0.1578
2022-10-16 16:44:14,018 - mmdet - INFO - Epoch [1][200/1337]	lr: 3.986e-04, eta: 1:45:49, time: 0.471, data_time: 0.011, memory: 14415, loss_rpn_cls: 0.0291, loss_rpn_bbox: 0.0092, loss_cls: 0.0681, acc: 98.2598, loss_bbox: 0.0668, loss: 0.1732
2022-10-16 16:44:37,7

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 764/764, 18.6 task/s, elapsed: 41s, ETA:     0s

2022-10-16 16:53:50,192 - mmdet - INFO - Evaluating bbox...


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 764/764, 18.3 task/s, elapsed: 42s, ETA:     0s

2022-10-16 18:35:37,772 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...


2022-10-16 18:35:37,988 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.718
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.979
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.867
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.410
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.733
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.765
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.765
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.765
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.483
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.04s).


In [7]:
test_from_config('work_dirs/retinanet_r50_fpn_2x_acino/retinanet_r50_fpn_2x_acino.py',
                 'work_dirs/retinanet_r50_fpn_2x_acino/latest.pth')

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
load checkpoint from local path: work_dirs/retinanet_r50_fpn_2x_acino/latest.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1528/1528, 11.0 task/s, elapsed: 139s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.77s).
Accumulating evaluation results...
DONE (t=0.29s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.732
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.966
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.911
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.463
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.746
 Average Recall     (AR) @[ IoU=0.50:0.

In [4]:
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.cnn import fuse_conv_bn
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)

from mmdet.apis import multi_gpu_test, single_gpu_test
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.utils import (build_ddp, build_dp, compat_cfg, get_device,
                         replace_cfg_vals, setup_multi_processes,
                         update_data_root)

def test_from_config(config,ckpt):
    out=False
    eval='bbox'
    format_only=False
    show_dir=None
    show_score_thr=0.3
    cfg = Config.fromfile(config)
    # replace the ${key} with the value of cfg.key
    cfg = replace_cfg_vals(cfg)

    # update data root according to MMDET_DATASETS
    update_data_root(cfg)
    cfg = compat_cfg(cfg)

    # set multi-process settings
    setup_multi_processes(cfg)
    if cfg.model.get('neck'):
        if isinstance(cfg.model.neck, list):
            for neck_cfg in cfg.model.neck:
                if neck_cfg.get('rfp_backbone'):
                    if neck_cfg.rfp_backbone.get('pretrained'):
                        neck_cfg.rfp_backbone.pretrained = None
        elif cfg.model.neck.get('rfp_backbone'):
            if cfg.model.neck.rfp_backbone.get('pretrained'):
                cfg.model.neck.rfp_backbone.pretrained = None
                
    cfg.device = get_device()
    distributed = False
    test_dataloader_default_args = dict(
        samples_per_gpu=1, workers_per_gpu=2, dist=distributed, shuffle=False)
    
    if isinstance(cfg.data.test, dict):
        cfg.data.test.test_mode = True
        if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
            # Replace 'ImageToTensor' to 'DefaultFormatBundle'
            cfg.data.test.pipeline = replace_ImageToTensor(
                cfg.data.test.pipeline)
    elif isinstance(cfg.data.test, list):
        for ds_cfg in cfg.data.test:
            ds_cfg.test_mode = True
        if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
            for ds_cfg in cfg.data.test:
                ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

    test_loader_cfg = {
        **test_dataloader_default_args,
        **cfg.data.get('test_dataloader', {})
    }

    rank, _ = get_dist_info()
    
    if cfg.work_dir is not None and rank == 0:
        mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
        timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
        json_file = osp.join(cfg.work_dir, f'eval_{timestamp}.json')

    # build the dataloader
    dataset = build_dataset(cfg.data.test)
    data_loader = build_dataloader(dataset, **test_loader_cfg)

    # build the model and load checkpoint
    cfg.model.train_cfg = None
    model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
    fp16_cfg = cfg.get('fp16', None)
    if fp16_cfg is not None:
        wrap_fp16_model(model)
    checkpoint = load_checkpoint(model, ckpt, map_location='cpu')
    if fuse_conv_bn:
        model = fuse_conv_bn(model)
    # old versions did not save class info in checkpoints, this walkaround is
    # for backward compatibility
    if 'CLASSES' in checkpoint.get('meta', {}):
        model.CLASSES = checkpoint['meta']['CLASSES']
    else:
        model.CLASSES = dataset.CLASSES

    if not distributed:
        model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)
        outputs = single_gpu_test(model, data_loader, False, None,
                                  show_score_thr)
    else:
        model = build_ddp(
            model,
            cfg.device,
            device_ids=[int(os.environ['LOCAL_RANK'])],
            broadcast_buffers=False)
        outputs = multi_gpu_test(
            model, data_loader, None, True
            or cfg.evaluation.get('gpu_collect', False))

    rank, _ = get_dist_info()
    if rank == 0:
        if out:
            print(f'\nwriting results to {out}')
            mmcv.dump(outputs, out)
        kwargs = {}
        if format_only:
            dataset.format_results(outputs, **kwargs)
        if eval:
            eval_kwargs = cfg.get('evaluation', {}).copy()
            # hard-code way to remove EvalHook args
            for key in [
                    'interval', 'tmpdir', 'start', 'gpu_collect', 'save_best',
                    'rule', 'dynamic_intervals'
            ]:
                eval_kwargs.pop(key, None)
            eval_kwargs.update(dict(metric=eval, **kwargs))
            metric = dataset.evaluate(outputs, **eval_kwargs)
            print(metric)
            metric_dict = dict(config=config, metric=metric)
            if cfg.work_dir is not None and rank == 0:
                mmcv.dump(metric_dict, json_file)

In [40]:

def test_detector(confg):
    evaluate='bbox'
    checkpoint=cfg.work_dir+'/latest.pth'
    out=None
    fuse_convbn=True
    show=False
    distributed=False

    show_dir=None#cfg.work_dir ##
    show_score_thr=0.7
    tmpdir=None
    gpu_collect=True
    eval_options=None
    format_only=None
    launcher="none"
    gpu_ids=cfg.gpu_ids

    cfg = replace_cfg_vals(cfg)

    # update data root according to MMDET_DATASETS
    update_data_root(cfg)

    cfg = compat_cfg(cfg)

    # set multi-process settings
    setup_multi_processes(cfg)

    # set cudnn_benchmark
    if cfg.get('cudnn_benchmark', False):
        torch.backends.cudnn.benchmark = True

    if 'pretrained' in cfg.model:
        cfg.model.pretrained = None
    elif 'init_cfg' in cfg.model.backbone:
        cfg.model.backbone.init_cfg = None

    if cfg.model.get('neck'):
        if isinstance(cfg.model.neck, list):
            for neck_cfg in cfg.model.neck:
                if neck_cfg.get('rfp_backbone'):
                    if neck_cfg.rfp_backbone.get('pretrained'):
                        neck_cfg.rfp_backbone.pretrained = None
        elif cfg.model.neck.get('rfp_backbone'):
            if cfg.model.neck.rfp_backbone.get('pretrained'):
                cfg.model.neck.rfp_backbone.pretrained = None


    cfg.device = get_device()
    # init distributed env first, since logger depends on the dist info.
    if launcher == 'none':
        distributed = False
    else:
        distributed = True
        init_dist(launcher, **cfg.dist_params)

    test_dataloader_default_args = dict(
        samples_per_gpu=1, workers_per_gpu=2, dist=distributed, shuffle=False)

    if isinstance(cfg.data.test, dict):
        cfg.data.test.test_mode = True
        if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
            # Replace 'ImageToTensor' to 'DefaultFormatBundle'
            cfg.data.test.pipeline = replace_ImageToTensor(
                cfg.data.test.pipeline)
    elif isinstance(cfg.data.test, list):
        for ds_cfg in cfg.data.test:
            ds_cfg.test_mode = True
        if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
            for ds_cfg in cfg.data.test:
                ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

    test_loader_cfg = {
        **test_dataloader_default_args,
        **cfg.data.get('test_dataloader', {})
    }

    rank, _ = get_dist_info()
    # allows not to create
    if cfg.work_dir is not None and rank == 0:
        mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
        timestamp = time.strftime('%d-%m-', time.localtime())+"ep"+str(cfg.total_epochs)
        json_file = osp.join(cfg.work_dir, f'eval_{timestamp}.json')

    # build the dataloader
    dataset = build_dataset(cfg.data.test)
    data_loader = build_dataloader(dataset, **test_loader_cfg)

    # build the model and load checkpoint
    cfg.model.train_cfg = None
    model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
    fp16_cfg = cfg.get('fp16', None)
    if fp16_cfg is not None:
        wrap_fp16_model(model)
    checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')
    if fuse_convbn:
        model = fuse_conv_bn(model)
    # old versions did not save class info in checkpoints, this walkaround is
    # for backward compatibility
    if 'CLASSES' in checkpoint.get('meta', {}):
        model.CLASSES = checkpoint['meta']['CLASSES']
    else:
        model.CLASSES = dataset.CLASSES

    if not distributed:
        model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)
        outputs = single_gpu_test(model, data_loader, show, show_dir,
                                  show_score_thr)
    else:
        model = build_ddp(
            model,
            cfg.device,
            device_ids=[int(os.environ['LOCAL_RANK'])],
            broadcast_buffers=False)
        outputs = multi_gpu_test(
            model, data_loader, tmpdir, gpu_collect
            or cfg.evaluation.get('gpu_collect', False))

    rank, _ = get_dist_info()
    if rank == 0:
        if out:
            print(f'\nwriting results to {out}')
            mmcv.dump(outputs, out)
        kwargs = {} if eval_options is None else eval_options
        if format_only:
            dataset.format_results(outputs, **kwargs)
        if evaluate:
            eval_kwargs = cfg.get('evaluation', {}).copy()
            # hard-code way to remove EvalHook args
            for key in [
                    'interval', 'tmpdir', 'start', 'gpu_collect', 'save_best',
                    'rule', 'dynamic_intervals'
            ]:
                eval_kwargs.pop(key, None)
            eval_kwargs.update(dict(metric=evaluate, **kwargs))
            metric = dataset.evaluate(outputs, **eval_kwargs)
            print(metric)
            metric_dict = dict(config=config, metric=metric)
            if cfg.work_dir is not None and rank == 0:
                mmcv.dump(metric_dict, json_file)